In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np 
from scipy import signal 
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa

2024-01-24 14:47:26.172094: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-24 14:47:26.217414: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 14:47:26.217442: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 14:47:26.217469: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-24 14:47:26.226010: I tensorflow/core/platform/cpu_feature_g

In [4]:
x_train = np.load('../BraiNeoCare/Datasets/GAT/traindata.npy',mmap_mode='r')
y_train = np.load('../BraiNeoCare/Datasets/GAT/trainlabels.npy',mmap_mode='r')
x_test = np.load('../BraiNeoCare/Datasets/GAT/testdata.npy',mmap_mode='r')
y_test = np.load('../BraiNeoCare/Datasets/GAT/testlabels.npy',mmap_mode='r')

In [5]:
mean=x_train.mean()
std=x_train.std()
x_train=(x_train-mean)/std
x_test=(x_test-mean)/std
# x1=(x1-mean)/std

x_train=np.expand_dims(x_train,axis=-1)
x_test=np.expand_dims(x_test,axis=-1)
# x1=np.expand_dims(x1,axis=-1)

np.random.seed(42)
train_indices = np.arange(x_train.shape[0])
np.random.shuffle(train_indices)
x_train = x_train[train_indices]
y_train = y_train[train_indices]
# test_indices = np.arange(x_test.shape[0])   
# np.random.shuffle(test_indices)
# x_test = x_test[test_indices]
# y_test = y_test[test_indices]

In [6]:
channel_names=["Fp1-T3","T3-O1","Fp1-C3","C3-O1","Fp2-C4","C4-O2","Fp2-T4","T4-O2","T3-C3","C3-Cz","Cz-C4","C4-T4"]
indices =[[r,i] for r,c1 in enumerate(channel_names) for i,c2 in enumerate(channel_names) if (c1.split("-")[0]==c2.split("-")[1] or c1.split("-")[1]==c2.split("-")[1] 
          or c1.split("-")[0]==c2.split("-")[0] or c1.split("-")[1]==c2.split("-")[0])]
adj=np.zeros((12,12))
for i in indices:
    adj[i[0]][i[1]]=1
adj=tf.constant(adj,dtype=tf.float32)
adj

2024-01-24 14:49:20.148014: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14947 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-01-24 14:49:20.148769: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14946 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:73:00.0, compute capability: 7.5


<tf.Tensor: shape=(12, 12), dtype=float32, numpy=
array([[1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1.],
       [1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1.]], dtype=float32)>

In [8]:
class GATLayer(layers.Layer):

    def __init__(self,output_dim):
        super(GATLayer, self).__init__()
        self.output_dim = output_dim
        self.LeakyReLU = layers.LeakyReLU(alpha=0.2)
    
    def build(self, input_shape):
        self.W = self.add_weight(name='W',shape=(input_shape[-1], self.output_dim), initializer='random_normal',trainable=True)
        self.a = self.add_weight(name='a',shape=(2*self.output_dim, 1), initializer='random_normal',trainable=True)
    
    def call(self,input,adj):
        H= tf.matmul(input, self.W)
        h1=tf.tile(tf.expand_dims(H, axis=1), [1,12,1,1])
        h2=tf.tile(tf.expand_dims(H, axis=2), [1,1,12,1])
        result =tf.concat([h1 , h2], axis=-1)
        e=self.LeakyReLU(tf.squeeze(tf.matmul(result, self.a),axis=-1))
        zero_mat=-1e20*tf.zeros_like(e)
        msked_e=tf.where(adj==1,e,zero_mat)
        alpha=tf.nn.softmax(msked_e,axis=-1)
        HPrime=tf.matmul(alpha,H)
        return tf.nn.elu(HPrime)


Input= keras.Input(shape=(12,384,1))

x= layers.Conv2D(8,(1,3),activation='swish',padding='same')(Input)
y= layers.Conv2D(8,(1,5),activation='swish',padding='same')(Input)
x= layers.add([x,y])
x= layers.MaxPooling2D((1,2))(x)
x= layers.BatchNormalization()(x)
x= layers.SpatialDropout2D(0.1)(x)

x= layers.Conv2D(32,(1,3),activation='swish',padding='same')(x)
y= layers.Conv2D(32,(1,5),activation='swish',padding='same')(x)
x= layers.add([x,y])
x= layers.MaxPooling2D((1,2))(x)
x= layers.BatchNormalization()(x)
x= layers.SpatialDropout2D(0.1)(x)

x= layers.Conv2D(8,(1,3),activation='swish',padding='same')(x)
y= layers.Conv2D(8,(1,5),activation='swish',padding='same')(x)
x= layers.add([x,y])
x= layers.MaxPooling2D((1,2))(x)
x= layers.BatchNormalization()(x)
x= layers.SpatialDropout2D(0.1)(x)

x= layers.Conv2D(1,(1,3),activation='swish',padding='same')(x)
y= layers.Conv2D(1,(1,5),activation='swish',padding='same')(x)
x= layers.add([x,y])
x= layers.MaxPooling2D((1,2))(x)
x= layers.Reshape((12,24))(x)

x= GATLayer(37)(x,adj)
x= GATLayer(32)(x,adj)
x= GATLayer(16)(x,adj)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x= layers.Dense(32,activation='swish')(x)
x= layers.Dropout(0.1)(x)
x = layers.Dense(16,activation='swish')(x)
x = layers.Dense(1,activation='sigmoid')(x)

model = keras.Model(inputs=Input, outputs=x)

In [9]:
optimizer=keras.optimizers.Adam(learning_rate=0.002,weight_decay=0.0025)
loss=keras.losses.BinaryFocalCrossentropy(from_logits=False,gamma=2,alpha=0.4,apply_class_balancing=True)
# loss=keras.losses.BinaryCrossentropy(from_logits=False)  
kappa=tfa.metrics.CohenKappa(num_classes=2)
fp=keras.metrics.FalsePositives()
tn=keras.metrics.TrueNegatives()
precision = keras.metrics.Precision()
recall = keras.metrics.Recall()
AUROC = keras.metrics.AUC(curve='ROC', name = 'AUROC')
AUPRC = keras.metrics.AUC(curve='PR', name = 'AUPRC')
model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy', AUROC, AUPRC,fp,tn, precision, recall,kappa])    

In [10]:
checkpoint_path = "GAT_model_correct_5/cp_{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path) 
cp_callback=keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=False,verbose=0,save_best_only=True,monitor='val_accuracy')  
history=model.fit(x_train,y_train,epochs=200,batch_size=512,verbose=1,validation_data=(x_test,y_test),callbacks=[cp_callback])

Epoch 1/200


2024-01-24 14:54:24.597337: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/spatial_dropout2d_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-01-24 14:54:25.348616: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-01-24 14:54:28.586442: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb7dc37b330 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-24 14:54:28.586477: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2024-01-24 14:54:28.586482: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Quadro RTX 5000, Compute Capability 7.5
2024-01-24 14:54:28.591543: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reprod

213/213 [==============================] - ETA: 0s - loss: 0.0665 - accuracy: 0.7316 - AUROC: 0.7244 - AUPRC: 0.6316 - false_positives: 1326.0000 - true_negatives: 71026.0000 - precision: 0.8646 - recall: 0.2332 - cohen_kappa: 0.2626INFO:tensorflow:Assets written to: GAT_model_correct_5/cp_0001.ckpt/assets


INFO:tensorflow:Assets written to: GAT_model_correct_5/cp_0001.ckpt/assets


213/213 [==============================] - 42s 139ms/step - loss: 0.0665 - accuracy: 0.7316 - AUROC: 0.7244 - AUPRC: 0.6316 - false_positives: 1326.0000 - true_negatives: 71026.0000 - precision: 0.8646 - recall: 0.2332 - cohen_kappa: 0.2626 - val_loss: 0.0638 - val_accuracy: 0.7432 - val_AUROC: 0.8460 - val_AUPRC: 0.7787 - val_false_positives: 132.0000 - val_true_negatives: 14173.0000 - val_precision: 0.8983 - val_recall: 0.1841 - val_cohen_kappa: 0.2246
Epoch 2/200
213/213 [==============================] - 23s 109ms/step - loss: 0.0494 - accuracy: 0.8427 - AUROC: 0.8452 - AUPRC: 0.8207 - false_positives: 2348.0000 - true_negatives: 70004.0000 - precision: 0.9018 - recall: 0.5938 - cohen_kappa: 0.6136 - val_loss: 0.0858 - val_accuracy: 0.7245 - val_AUROC: 0.7543 - val_AUPRC: 0.6093 - val_false_positives: 152.0000 - val_true_negatives: 14153.0000 - val_precision: 0.8403 - val_recall: 0.1263 - val_cohen_kappa: 0.1515
Epoch 3/200
212/213 [============================>.] - ETA: 0s - loss:

...

213/213 [==============================] - 27s 128ms/step - loss: 0.0302 - accuracy: 0.9098 - AUROC: 0.9505 - AUPRC: 0.9356 - false_positives: 1877.0000 - true_negatives: 70475.0000 - precision: 0.9380 - recall: 0.7819 - cohen_kappa: 0.7886 - val_loss: 0.0450 - val_accuracy: 0.8921 - val_AUROC: 0.8959 - val_AUPRC: 0.8071 - val_false_positives: 340.0000 - val_true_negatives: 13965.0000 - val_precision: 0.9290 - val_recall: 0.7020 - val_cohen_kappa: 0.7277
Epoch 29/200
213/213 [==============================] - 23s 107ms/step - loss: 0.0300 - accuracy: 0.9116 - AUROC: 0.9512 - AUPRC: 0.9363 - false_positives: 1847.0000 - true_negatives: 70505.0000 - precision: 0.9392 - recall: 0.7862 - cohen_kappa: 0.7929 - val_loss: 0.0514 - val_accuracy: 0.8803 - val_AUROC: 0.8708 - val_AUPRC: 0.7738 - val_false_positives: 315.0000 - val_true_negatives: 13990.0000 - val_precision: 0.9299 - val_recall: 0.6597 - val_cohen_kappa: 0.6938
Epoch 30/200
213/213 [==============================] - 23s 107ms/ste

INFO:tensorflow:Assets written to: GAT_model_correct_5/cp_0053.ckpt/assets


213/213 [==============================] - 26s 123ms/step - loss: 0.0265 - accuracy: 0.9202 - AUROC: 0.9625 - AUPRC: 0.9499 - false_positives: 1689.0000 - true_negatives: 70663.0000 - precision: 0.9455 - recall: 0.8077 - cohen_kappa: 0.8139 - val_loss: 0.0517 - val_accuracy: 0.8923 - val_AUROC: 0.9165 - val_AUPRC: 0.8190 - val_false_positives: 373.0000 - val_true_negatives: 13932.0000 - val_precision: 0.9232 - val_recall: 0.7081 - val_cohen_kappa: 0.7294
Epoch 54/200
213/213 [==============================] - 23s 106ms/step - loss: 0.0265 - accuracy: 0.9202 - AUROC: 0.9625 - AUPRC: 0.9499 - false_positives: 1628.0000 - true_negatives: 70724.0000 - precision: 0.9473 - recall: 0.8059 - cohen_kappa: 0.8137 - val_loss: 0.0548 - val_accuracy: 0.8780 - val_AUROC: 0.9098 - val_AUPRC: 0.8127 - val_false_positives: 304.0000 - val_true_negatives: 14001.0000 - val_precision: 0.9313 - val_recall: 0.6505 - val_cohen_kappa: 0.6870
Epoch 55/200
213/213 [==============================] - 23s 107ms/ste

INFO:tensorflow:Assets written to: GAT_model_correct_5/cp_0061.ckpt/assets


213/213 [==============================] - 27s 127ms/step - loss: 0.0260 - accuracy: 0.9220 - AUROC: 0.9642 - AUPRC: 0.9517 - false_positives: 1656.0000 - true_negatives: 70696.0000 - precision: 0.9468 - recall: 0.8121 - cohen_kappa: 0.8182 - val_loss: 0.0477 - val_accuracy: 0.8955 - val_AUROC: 0.9267 - val_AUPRC: 0.8298 - val_false_positives: 357.0000 - val_true_negatives: 13948.0000 - val_precision: 0.9270 - val_recall: 0.7160 - val_cohen_kappa: 0.7379
Epoch 62/200
213/213 [==============================] - 23s 108ms/step - loss: 0.0260 - accuracy: 0.9226 - AUROC: 0.9638 - AUPRC: 0.9517 - false_positives: 1641.0000 - true_negatives: 70711.0000 - precision: 0.9474 - recall: 0.8136 - cohen_kappa: 0.8198 - val_loss: 0.0594 - val_accuracy: 0.8651 - val_AUROC: 0.9118 - val_AUPRC: 0.8142 - val_false_positives: 275.0000 - val_true_negatives: 14030.0000 - val_precision: 0.9329 - val_recall: 0.6038 - val_cohen_kappa: 0.6483
Epoch 63/200
213/213 [==============================] - 23s 107ms/ste

In [ ]:
with open("history_GAT_rem_art_3.jason", 'w') as f:
    pd.DataFrame(history.history).to_json(f)

In [ ]:
# Plot the training and validation loss and accuracy for each epoch for 4s partitions
with open('history_GAT_rem_art_3.jason','r') as f:
    history_1 = pd.read_json(f)

# with open('history_GAT_rem_art.jason','r') as f:
#     history_2 = pd.read_json(f)

metrics=['accuracy','val_accuracy','loss','val_loss','val_AUROC','val_AUPRC','val_precision_1','val_recall_1','recall_1','precision_1','AUROC','AUPRC']

epochs = range(1, 201)

fig,ax=plt.subplots(4,3,figsize=(20,20))

for r in range(12):
    ax[r//3][r%3].plot(epochs,history_1[metrics[r]],label=metrics[r],color='r')
    # ax[r//3][r%3].plot(epochs,history_2[metrics[r]],label=metrics[r],color='g')
    ax[r//3][r%3].set_title(metrics[r])
    ax[r//3][r%3].set_xlabel('Epochs')
    # ax[r//3][r%3].axhline(y=0.975, color='r', linestyle='-')
    ax[r//3][r%3].axvline(x=197, color='b', linestyle='-')
    ax[r//3][r%3].grid()
fig.tight_layout()
plt.show()

In [ ]:
history_1['val_accuracy'].max() 